In [1]:
import casadi as ca
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


# Parameter

In [7]:
S=2170.0;
CBAR=17.5;
MASS=5.0E3; 
IYY= 4.1E6;
TSTAT=6.0E4;
DTDV = -38.0;
ZE = 2.0;
CDCLS= 0.042;
CLA = 0.085;
CMA = -0.022; 
CMDE = -0.016;  
CMQ =-16.0;
CMADOT= -6.0;
CLADOT= 0.0;
RTOD = 57.29578;
GD=32.17;
R0 = 2.377e-5;

In [8]:
thtl = ca.SX.sym('thtl')
elev = ca.SX.sym('elev')
xcg = ca.SX.sym('xcg')
land = ca.SX.sym('land')
VT = ca.SX.sym('VT')
alhpa = RTOD * ca.SX.sym('alpha')
theta = ca.SX.sym('theta')
Q = ca.SX.sym('Q')
H = ca.SX.sym('H')

In [10]:
tfac = 1.0 - 0.703e-5 * H
T = 519.0 * tfac
rho = R0 * (tfac ** 4.14)
qbar = 0.5*rho*VT*VT
mach = VT / np.sqrt(1.4*1716.3*T)

In [ ]:
qs = qbar * S;
salp = np.sin()